# Adversaraial training

In [10]:
import os
import json
import PIL.Image
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision

from src.data.ImageNet100ClassesDataset import ImageNet100ClassesDataset, prepare_dataloaders_ImageNet100ClassesDataset
from src.attacks.attacks import FastGradientSign, ProjectedGradientDescent
from src.training.Trainer import Trainer

In [2]:
def seed_everything(seed_value=4995):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

In [3]:
root_dir = 'Data/imagenet100classes'

In [4]:
config = {
    'lr': 0.001,
    'batch_size': 64,
    'weight_decay': 0.01
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def load_model(num_classes = 100, model_path = None, to_cuda = True):
    if not model_path:
        model = torchvision.models.resnet18(pretrained = True)
        input_feat = model.fc.in_features
        
        model.fc = nn.Linear(input_feat, num_classes)
        loaded_state_dict = False
    
    else:
        print("Loaded", model_path)
        model = torchvision.models.resnet18()
        input_feat = model.fc.in_features
        model.fc = nn.Linear(input_feat, num_classes)
        loaded_model = torch.load(model_path)
        model.load_state_dict(loaded_model['model_state_dict'])
        loaded_state_dict = True
        
    if to_cuda:
        model = model.to('cuda')
        
    return model, loaded_state_dict

## Train Classification head

In [6]:
resnet, loaded_state_dict = load_model()

for name, param in resnet.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False

In [7]:
train_loader, val_loader = prepare_dataloaders_ImageNet100ClassesDataset(root_dir, batch_size = config['batch_size'])

dataloaders = {
    'train': train_loader,
    'val': val_loader
}

loss_fn = nn.CrossEntropyLoss()

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = config['lr'], weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3,5], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

In [8]:
classes = list(train_loader.dataset.class_name_dict.values())

In [12]:
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 20, epsilon = 0.25, return_logits=False)

In [19]:
trainer = Trainer(resnet, loss_fn, classes, training_params, DEVICE, num_epochs = 6, model_name = 'resnet_100_imagenet_adv_training', save_model = True, model_dir = 'models', adversarial_training = True, adversarial_attack = pgd)

In [14]:
trainer.train()

Current Epoch: 0
Train Loop:
Batch: 0 of 2032. Loss: 8.628345489501953. Mean so far: 8.628345489501953. Mean of 100: 8.628345489501953
Batch: 100 of 2032. Loss: 5.833389759063721. Mean so far: 6.616345235616854. Mean of 100: 6.596225233078003
Batch: 200 of 2032. Loss: 5.336579322814941. Mean so far: 6.087421369789845. Mean of 100: 5.553208265304566
Batch: 300 of 2032. Loss: 5.201359748840332. Mean so far: 5.825791656772956. Mean of 100: 5.299915933609009
Batch: 400 of 2032. Loss: 5.165585517883301. Mean so far: 5.660966002733035. Mean of 100: 5.164840784072876
Batch: 500 of 2032. Loss: 5.195232391357422. Mean so far: 5.548921068271477. Mean of 100: 5.099620881080628
Batch: 600 of 2032. Loss: 5.054399490356445. Mean so far: 5.473045780734095. Mean of 100: 5.092910590171814
Batch: 700 of 2032. Loss: 5.10078239440918. Mean so far: 5.412950577647472. Mean of 100: 5.0517784070968625
Batch: 800 of 2032. Loss: 5.020207405090332. Mean so far: 5.3656477195939765. Mean of 100: 5.034054684638977


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
trainer.train()

Val Loop:
Batch: 0 of 79. Loss: 4.799765586853027. Mean so far: 4.799765586853027. Mean of 100: 4.799765586853027
Validation loss: 5.03048982499521. Val measure: 0.0074
Validation loop took 297.9932527542114
Current Epoch: 0
Eval  Model:  resnet_100_imagenet_adv_training . Acc:  0.0074 . Avg loss: 5.03048982499521
Train Model:  resnet_100_imagenet_adv_training . Acc 0.005715384615384615 Avg loss: 5.153614494040257
Current acc:  0.0074 5.03048982499521 at epoch 0
Accuracy of chambered nautilus, pearly nautilus, nautilus                         :  0.0
Accuracy of harvestman, daddy longlegs, Phalangium opilio                         :  0.0
Accuracy of macaw                                                                 :  0.0
Accuracy of bittern                                                               :  0.0
Accuracy of electric ray, crampfish, numbfish, torpedo                            :  0.0
Accuracy of drake                                                                 :  0.0

## Fine-tune network

In [6]:
resnet, loaded_state_dict = load_model(model_path = 'models/resnet_100_imagenet_adv_training.pt')

for name, param in resnet.named_parameters():
    param.requires_grad = True

Loaded models/resnet_100_imagenet.pt


In [7]:
train_loader, val_loader = prepare_dataloaders_ImageNet100ClassesDataset(root_dir, batch_size = config['batch_size'])

dataloaders = {
    'train': train_loader,
    'val': val_loader
}

loss_fn = nn.CrossEntropyLoss()

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = 0.00001, weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3,5], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

In [8]:
classes = list(train_loader.dataset.class_name_dict.values())

In [9]:
trainer_fine_tune = Trainer(resnet, loss_fn, classes, training_params, DEVICE, num_epochs = 6, model_name = 'resnet_100_imagenet_adv_training_fine_tuned', save_model = True, model_dir = 'models', adversarial_training = True, adversarial_attack = pgd)

In [10]:
trainer_fine_tune.train()

Current Epoch: 0
Train Loop:
Batch: 0 of 2032. Loss: 0.7929084300994873. Mean so far: 0.7929084300994873. Mean of 100: 0.7929084300994873
Batch: 100 of 2032. Loss: 0.5967435240745544. Mean so far: 0.6240629769197785. Mean of 100: 0.6223745223879814
Batch: 200 of 2032. Loss: 0.6304162740707397. Mean so far: 0.6015580545610456. Mean of 100: 0.5788280829787255
Batch: 300 of 2032. Loss: 0.5305232405662537. Mean so far: 0.59579783123593. Mean of 100: 0.5842197823524475
Batch: 400 of 2032. Loss: 0.4131864607334137. Mean so far: 0.5842891300556963. Mean of 100: 0.5496479395031929
Batch: 500 of 2032. Loss: 0.5630130171775818. Mean so far: 0.577194420520417. Mean of 100: 0.548744635283947
Batch: 600 of 2032. Loss: 0.3642585575580597. Mean so far: 0.5735786809104056. Mean of 100: 0.5554638254642487
Batch: 700 of 2032. Loss: 0.47939738631248474. Mean so far: 0.5714079341857817. Mean of 100: 0.5583617463707924
Batch: 800 of 2032. Loss: 0.5330355167388916. Mean so far: 0.5695001342323389. Mean of 1